# Running a SCF

In [ ]:
import Auto_baby

In [ ]:
print(Auto_baby)

In [ ]:
silicon_experiment = Auto_baby.Experiment(material_dir='/home/student/zylu/silicon_demo')

In [ ]:
silicon_experiment.ensure_directories()

In [ ]:
silicon_experiment.ensure_required_files()

In [ ]:
silicon_experiment.scf_calculation()

In [ ]:
silicon_experiment.dos()

In [ ]:
silicon_experiment.band_structure()

# Automation with FireWorks

## Download POSCAR files from MP offical repo and save them to `material_path/POSCAR`

In [ ]:
from mp_api.client import MPRester

with MPRester("5ypNyStQsaDUA2neNLD17P8v2KNB241U") as mpr:
    
    structure = mpr.get_structure_by_material_id("mp-149") 
    structure.to(fmt="poscar")  
    structure.to_file("/home/student/zylu/demo/1_scf/POSCAR")  
    print(structure)

## WorkFlows

In [1]:
from fireworks import Firework, LaunchPad, FileTransferTask, PyTask, Workflow, ScriptTask
from fireworks.core.rocket_launcher import rapidfire
import Auto_baby

def Final_Calculation(material_path:str = ''):
    if not material_path: raise ValueError("material_path must be provided.")


    launchpad = LaunchPad()
    launchpad.reset('', require_password=False)
    

    _submit_SCF_Task_ = PyTask(func = "Auto_baby.scf_ready",
                               args=[material_path])



    _transfer_CONTCAR_of_SCF_to_DOS = ScriptTask({'script': f'cp {material_path}/1_scf/CONTCAR {material_path}/2_dos/POSCAR'})

    _transfer_POTCAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                       'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                  'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    _transfer_WAVECAR_of_SCF_to_DOS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/WAVECAR', 
                                                       'dest': f'{material_path}/2_dos'}], 'mode': 'copy'})
    

    _submit_DOS_Task_ = PyTask(func= "Auto_baby.dos_ready",
                               args=[material_path])



    _transfer_CONTCAR_of_SCF_to_BS = ScriptTask({'script': f'cp {material_path}/1_scf/CONTCAR {material_path}/3_bs/POSCAR'})
    _transfer_POTCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/POTCAR', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    _transfer_JOB_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/job.sh', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    _transfer_CHGCAR_of_SCF_to_BS = FileTransferTask({'files': [{'src': f'{material_path}/1_scf/CHGCAR', 
                                                    'dest': f'{material_path}/3_bs'}], 'mode': 'copy'})
    

    _submit_BS_Task_ = PyTask(func = "Auto_baby.bs_ready",
                             args=[material_path])



    scf_calculation = Firework(
        tasks=[_submit_SCF_Task_],
        name="SCF Calculation" 
    )

    Transfer_SCF_files_to_DOS = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_DOS, _transfer_POTCAR_of_SCF_to_DOS, _transfer_JOB_of_SCF_to_DOS, _transfer_WAVECAR_of_SCF_to_DOS,_submit_DOS_Task_],
        name="Transfer SCF files to DOS",
        parents=[scf_calculation]
    )

    BAND_structure_calculation = Firework(
        tasks=[_transfer_CONTCAR_of_SCF_to_BS, _transfer_POTCAR_of_SCF_to_BS, _transfer_JOB_of_SCF_to_BS, _transfer_CHGCAR_of_SCF_to_BS, _submit_BS_Task_],
        name="Band Structure Calculation",
        parents=[scf_calculation]
    )



    wf = Workflow([scf_calculation,Transfer_SCF_files_to_DOS,BAND_structure_calculation, ])
    launchpad.add_wf(wf)


    rapidfire(launchpad)

In [2]:
Final_Calculation(material_path='/home/student/zylu/silicon_demo')


2025-11-14 14:41:24,279 INFO Performing db tune-up


2025-11-14 14:41:24,295 INFO LaunchPad was RESET.
2025-11-14 14:41:24,297 INFO Added a workflow. id_map: {-3: 1, -2: 2, -1: 3}
2025-11-14 14:41:24,381 INFO Created new dir /home/student/zylu/fireworks/launcher_2025-11-14-05-41-24-380923
2025-11-14 14:41:24,381 INFO Launching Rocket
2025-11-14 14:41:24,388 INFO RUNNING fw_id: 3 in directory: /home/student/zylu/fireworks/launcher_2025-11-14-05-41-24-380923
2025-11-14 14:41:24,392 INFO Task started: PyTask.
All required calculation inputs and job scripts are present where needed.
Current SLURM job state: UNKNOWN
Current SLURM job state: PENDING
Current SLURM job state: RUNNING
Current SLURM job state: RUNNING
Current SLURM job state: COMPLETED
Final SLURM job state: COMPLETED
2025-11-14 14:41:28,899 INFO Task completed: PyTask
2025-11-14 14:41:28,912 INFO Rocket finished
2025-11-14 14:41:28,915 INFO Created new dir /home/student/zylu/fireworks/launcher_2025-11-14-05-41-28-915101
2025-11-14 14:41:28,915 INFO Launching Rocket
2025-11-14 14:

## PostProcessing (py4vasp)

In [ ]:
import py4vasp
mycalc = py4vasp.Calculation.from_file("/home/student/zylu/silicon_demo/2_dos/vaspout.h5")

mycalc.dos.plot()

mycalc.dos.to_image("dos_plot.png")


In [ ]:
import py4vasp
mycalc = py4vasp.Calculation.from_file("/home/student/zylu/silicon_demo/3_bs/vaspout.h5")

mycalc.band.plot()